In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [18]:
data_dict = pd.read_csv("data_dictionary.csv")

In [38]:
visit = pd.read_csv('C:\\Users\\ASUS\\Desktop\\covid_pure\\training\\visit_occurrence.csv',
                    usecols=["person_id", "visit_concept_id","visit_start_date", "visit_end_date"])
visit.head()

,person_id,visit_concept_id,visit_start_date,visit_end_date
0,42,9201,2011-06-18,2011-06-27
1,554,581479,2012-08-31,2012-08-31
2,889,9203,2016-01-10,2016-01-12
3,970,9201,2010-05-18,2010-05-23
4,568,32037,2019-06-21,2019-06-30


In [51]:
gs = pd.read_csv("C:\\Users\\ASUS\\Desktop\\covid_pure\\training\\goldstandard.csv")
gs.head()

,person_id,status
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0


In [56]:
gs[gs["status"] == 0]

,person_id,status
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
1246,1246,0.0
1247,1247,0.0
1248,1248,0.0
1249,1249,0.0


In [39]:
person = pd.read_csv('C:\\Users\\ASUS\\Desktop\\covid_pure\\training\\person.csv')
person.head()

,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
0,0,8532,1951,9,18,1951-09-18,8515,38003564,1811,NaN,NaN,NaN,F,0,NaN,10,NaN,29
1,1,8532,1934,8,15,1934-08-15,8552,38003563,7867,NaN,NaN,NaN,F,0,NaN,8,NaN,28
2,2,8507,1993,2,20,1993-02-20,8552,38003564,7807,NaN,NaN,NaN,M,0,NaN,26,NaN,19
3,3,8532,1974,8,7,1974-08-07,8527,38003564,3498,NaN,NaN,NaN,F,0,NaN,30,NaN,29
4,4,8532,1942,5,9,1942-05-09,8527,38003563,4350,NaN,NaN,NaN,F,0,NaN,30,NaN,28


In [40]:
visit['visit_start_date'] = pd.to_datetime(visit['visit_start_date'])
visit['visit_end_date'] = pd.to_datetime(visit['visit_end_date'])

In [42]:
visit_concept_ids = visit['visit_concept_id'].value_counts().index.to_list()
concept_ids = {}
for v_id in visit_concept_ids:
    value = data_dict[data_dict['concept_id'] == v_id]['concept_name'].values[0]
    concept_ids[v_id] = value
visit['visit_concept_id'] = visit['visit_concept_id'].map(concept_ids)

In [43]:
visit.head()

,person_id,visit_concept_id,visit_start_date,visit_end_date
0,42,Inpatient Visit,2011-06-18,2011-06-27
1,554,Ambulatory Rehabilitation Visit,2012-08-31,2012-08-31
2,889,Emergency Room Visit,2016-01-10,2016-01-12
3,970,Inpatient Visit,2010-05-18,2010-05-23
4,568,Intensive Care,2019-06-21,2019-06-30


In [61]:
last_visits = visit.sort_values(['visit_end_date'],ascending=False).groupby('person_id').head(1).reset_index(drop=True)

In [63]:
new_data = person.merge(last_visits, on="person_id")

In [65]:
new_data = new_data.merge(gs, on="person_id")
new_data

,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,...,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id,visit_concept_id,visit_start_date,visit_end_date,status
0,0,8532,1951,9,18,1951-09-18,8515,38003564,1811,NaN,...,F,0,NaN,10,NaN,29,Emergency Room Visit,2020-06-13,2020-06-19,0.0
1,1,8532,1934,8,15,1934-08-15,8552,38003563,7867,NaN,...,F,0,NaN,8,NaN,28,Intensive Care,2020-08-03,2020-08-09,0.0
2,2,8507,1993,2,20,1993-02-20,8552,38003564,7807,NaN,...,M,0,NaN,26,NaN,19,Emergency Room Visit,2020-04-25,2020-04-28,0.0
3,3,8532,1974,8,7,1974-08-07,8527,38003564,3498,NaN,...,F,0,NaN,30,NaN,29,Inpatient Visit,2020-07-25,2020-08-02,0.0
4,4,8532,1942,5,9,1942-05-09,8527,38003563,4350,NaN,...,F,0,NaN,30,NaN,28,Emergency Room Visit,2020-03-30,2020-04-04,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,1246,8507,1967,3,25,1967-03-25,8552,38003564,961,NaN,...,M,0,NaN,26,NaN,19,Emergency Room Visit,2020-06-18,2020-06-24,0.0
1247,1247,8532,1966,10,3,1966-10-03,8552,38003563,6111,NaN,...,F,0,NaN,26,NaN,28,Telehealth,2020-06-07,2020-06-10,0.0
1248,1248,8507,1983,3,27,1983-03-27,8552,38003564,2034,NaN,...,M,0,NaN,26,NaN,19,Intensive Care,2019-12-03,2019-12-03,0.0
1249,1249,8507,1939,7,3,1939-07-03,8527,38003564,3744,NaN,...,M,0,NaN,30,NaN,29,Telephone call to a patient,2020-02-10,2020-02-15,0.0


In [67]:
new_data[new_data['status'] == 1]['visit_concept_id'].value_counts()

Emergency Room Visit               22
Ambulatory Rehabilitation Visit    19
Intensive Care                     16
Telephone call to a patient        16
Inpatient Visit                    15
Telehealth                         11
Outpatient Visit                    8
Name: visit_concept_id, dtype: int64

In [68]:
new_data[new_data['status'] == 0]['visit_concept_id'].value_counts()

Emergency Room Visit               187
Telehealth                         184
Intensive Care                     177
Telephone call to a patient        171
Inpatient Visit                    163
Outpatient Visit                   136
Ambulatory Rehabilitation Visit    126
Name: visit_concept_id, dtype: int64